In [1]:
import sys 
sys.path.append('../..')

In [2]:
from main_utils import LLM_EmbedData

In [38]:
from datasets import load_dataset, Dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [12]:
data_demo = load_dataset("SetFit/SentEval-CR")

  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
data_demo['train'][:2]

{'text': ["many of our disney movies do n 't play on this dvd player .",
  "player has a problem with dual-layer dvd 's such as alias season 1 and season 2 ."],
 'label': [0, 0],
 'label_text': ['negative', 'negative']}

In [14]:
data = LLM_EmbedData(
    '../../datasets/product_title_embedding/wish-tahoe-openai-reversegen.yaml', 
    model_name='sentence-transformers/sentence-t5-base'
)

In [15]:
data.prepare_data()

In [16]:
ds = data.get_hf_dataset()

In [26]:
ds['train'][:2]

{'text_input': ["Embed product: Boys' Leather Belt, Ratchet Automatic Buckle Belt For Men",
  'Embed product: Boys Blue Light Blocking Glasses, Anti Blue Ray Computer Gaming Reading Glasses'],
 'text_output': ["Embed taxonomy: apparel accessories > boys' accessories > boys' belts",
  "Embed taxonomy: apparel accessories > boys' accessories > boys' glasses > boys' blue light blocking glasses"]}

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
le = LabelEncoder()

In [25]:
le.fit(sorted(list(set(ds['train'][:]['text_output'] + ds['val'][:]['text_output'] + ds['test'][:]['text_output']))))

LabelEncoder()

In [ ]:
{'text': ["many of our disney movies do n 't play on this dvd player .",
  "player has a problem with dual-layer dvd 's such as alias season 1 and season 2 ."],
 'label': [0, 0],
 'label_text': ['negative', 'negative']}

In [27]:
def transform(examples):
    outs = {}
    outs['text'] = [i.split('Embed product: ')[1] for i in examples['text_input']]
    outs['label'] = le.transform([i for i in examples['text_output']]).tolist()
    outs['label_text'] = [i.split('Embed taxonomy: ')[1] for i in examples['text_output']]
    return outs

In [28]:
ds.set_transform(transform)

In [29]:
ds['train'][:]

{'text': ["Boys' Leather Belt, Ratchet Automatic Buckle Belt For Men",
  'Boys Blue Light Blocking Glasses, Anti Blue Ray Computer Gaming Reading Glasses'],
 'label': [0, 1],
 'label_text': ["apparel accessories > boys' accessories > boys' belts",
  "apparel accessories > boys' accessories > boys' glasses > boys' blue light blocking glasses"]}

In [39]:
import pandas as pd

In [40]:
train_ds = Dataset.from_pandas(pd.DataFrame(ds['train'][:]))
val_ds = Dataset.from_pandas(pd.DataFrame(ds['val'][:]))

In [41]:
model = SetFitModel.from_pretrained("sentence-transformers/sentence-t5-base")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [42]:
trainer.train()
metrics = trainer.evaluate()

KeyboardInterrupt: 